In [1]:
from dotenv import load_dotenv
load_dotenv()

from config import Config
config = Config.load()

In [2]:
from langfuse.callback import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()

In [ ]:
from utils import prompt_loader
prompt_loader.list_prompts()

In [4]:
from agent.agents import AgentFactory
agent_factory = AgentFactory()

from langfuse.callback import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()

### Step1: Search News

In [5]:
initial_search_agent = await agent_factory.get_initial_search_agent()

In [6]:
initial_search_agent = await agent_factory.get_initial_search_agent()
response = await initial_search_agent.ainvoke({"messages": [{"role": "user", "content": "Start searching for news about the latest trends in AI and machine learning, provide maximum 2 articles."}]},
                                        config={"callbacks": [langfuse_handler]})

In [ ]:
response

In [ ]:
response["structured_response"].articles

### Filter News

In [6]:
from data_model.news import NewsArticleList
from agent.agents import AgentFactory
agent_factory = AgentFactory()

from langfuse.callback import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()

github_assistant = await agent_factory.get_github_assistant(specific_tool=["get_file_contents", "get_me"], response_format=NewsArticleList)

In [10]:
news_filter_prompt = await prompt_loader.get_prompt("news_filter").template.render_async(news=response["structured_response"].articles)

result = await github_assistant.ainvoke({"messages": [{"role": "user", "content": news_filter_prompt}]},
                                        config={"callbacks": [langfuse_handler]})

In [ ]:
result

In [ ]:
result["structured_response"]

### Agent

In [5]:
from agent.agents import AgentFactory

agent_factory = AgentFactory(verbose=True)

from langfuse.callback import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()

#### summarise agent


In [ ]:
search_and_summarise_agent = await agent_factory.get_search_and_summarise_agent()

In [ ]:
search_and_summarise_agent

In [25]:
class Article:
    title: str = "Anthropic Releases Claude 4 with Autonomous Functionality"
    source: str = "Anthropic"
    date: str = "2025-05-22"
    summary: str = "Anthropic has unveiled Claude 4, featuring two models: Claude Opus 4 and Claude Sonnet 4. Notably, Claude 4 can operate autonomously for extended periods, enhancing its utility in complex tasks."
    url: str = "https://www.anthropic.com/claude-4-release"


article = Article()
    
    

In [26]:
news_format = prompt_loader.get_prompt("news_format")
news = await news_format.template.render_async(title=article.title, 
                                            source=article.source, 
                                            date=article.date, 
                                            summary=article.summary,
                                            url_ref=article.url)

In [27]:
output = search_and_summarise_agent.invoke({"messages": [{"role": "user", "content": news}]}, 
             config={"callbacks": [langfuse_handler]})

In [ ]:
print(output["messages"][-1].content[0]["text"])

In [ ]:
print(output["structured_response"].file_name)

In [ ]:
print(output["structured_response"].markdown_file)

#### github agent

In [ ]:
github_assistant = await agent_factory.get_github_assistant(specific_tool=["get_file_contents", "get_me"])

In [ ]:
prompt = """
Can you show me to add .requirements.txt file with numpy, pandas, \
and langchain under github project ai-news-ignite, under my GitHub account (jeff52415), \
create a new branch, and push the changes to the new branch, then make a pull request to merge the changes into the main branch?
"""
output = await github_assistant.ainvoke({"messages": [{"role": "user", 
                                                       "content": prompt}]},
                                        config={"callbacks": [langfuse_handler]})

In [ ]:
print(output["messages"][-1].content)

### Multi-Agent

In [ ]:
class Article:
    title: str = "Anthropic Releases Claude 4 with Autonomous Functionality"
    source: str = "Anthropic"
    date: str = "2025-05-22"
    summary: str = "Anthropic has unveiled Claude 4, featuring two models: Claude Opus 4 and Claude Sonnet 4. Notably, Claude 4 can operate autonomously for extended periods, enhancing its utility in complex tasks."
    url: str = "https://www.anthropic.com/claude-4-release"


article = Article()
    
    
supervisor = await agent_factory.get_supervisor()

In [18]:
supervisor.debug = True

In [19]:
supervisor_prompt= prompt_loader.get_prompt("supervisor_agent_input")
supervisor_prompt = await supervisor_prompt.template.render_async(title=article.title, 
                                                            source=article.source, 
                                                            date=article.date, 
                                                            summary=article.summary,
                                                            url_ref=article.url)

In [ ]:
print(supervisor_prompt)

In [ ]:
output = await supervisor.ainvoke({"messages": [{"role": "user", "content": supervisor_prompt}]},
                                        config={"callbacks": [langfuse_handler], "recursion_limit": 40}, )
print(output["messages"][-1].content)

In [ ]:
output

In [ ]:
output["messages"][-1].content